## Hurdat2, Plotting Storm Alongside Drifter Tracks With Average Temperature
In the following example we will walk through how we can leverage the `clouddrift` library to plot drifter tracks colored by temperature recorded alongside hurricane storm tracks obtained from the HURDAT2 dataset. Simply put, the HURDAT2 dataset is a dataset that contains storm track data (including other measurements such as pressure, wind speed, etc...) for storms recorded from 1852 - 2022 across both the Pacific and Atlantic Ocean.

Lets proceed with loading in the datasets were interested in taking a look at.

We'll start with loading the drifter dataset. This will download and generate an aggregate version of the drifters dataset. Note this process may take some time ~15m and is only performed once unless the force flag is flipped to `True`

In [ ]:
import clouddrift
import xarray as xr
import os

fp = "aggregate_gdp6h_local.nc"
force = False

if not os.path.exists(fp) or force:
    clouddrift.adapters.gdp6h.to_raggedarray(n_random_id=10_000).to_netcdf(fp)
    drifter_ds = xr.load_dataset(fp, decode_times=True)
else:
    drifter_ds = xr.load_dataset(fp, decode_times=True)
drifter_ds

Lets now also load in the HURDAT2 storm dataset.

In [ ]:
storm_ds = clouddrift.datasets.hurdat2(decode_times=True)

Now lets say that we'd like to select a specific subset of this dataset; we can leverage the `subset` utility function provided through the `ragged` module which contains a library of helpful utility functions for working with the `RaggedArray` data structure.

As an example say you wanted a subset of the dataset for storms whose track lied within the Atlantic Ocean near the east coast of North America and was observed between August and October of 2020. You can leverage the `subset` function to help you achieve this by first defining the criteria:

In [ ]:
# import some helpful libraries
import numpy as np
from datetime import datetime

# Here the datasets variables are mapped to an (inclusive start and end) range
start_dt, end_dt = datetime(2020, 8, 1), datetime(2020, 10, 1)
criteria = dict(
    lat=(10, 50),
    lon=(-80, -20), 
    time=(
        np.datetime64(int(start_dt.timestamp()), "s"),
        np.datetime64(int(end_dt.timestamp()), "s")
    )
)

Lets use the `subset` function and apply the criteria to both datasets. Here we need to provide the row dimensions alias which is `traj` in both datasets.

In [ ]:
matching_storms = clouddrift.ragged.subset(storm_ds, criteria, row_dim_name="traj")
matching_storms

In [ ]:
matching_drifters = clouddrift.ragged.subset(drifter_ds, criteria, row_dim_name="traj")
matching_drifters

Now lets create our base plot where we will plot the drifter and storm trajectories.

In [ ]:
import cartopy.crs as ccrs  # cartopy for projecting our dataset onto different map projections
import matplotlib.pyplot as plt # is an standard plotting library
import matplotlib.animation as animation


DPI = 384
fig = plt.figure(figsize=(7.75, 4.75), dpi=DPI)

ax = fig.add_subplot(1, 1, 1, projection=ccrs.Mollweide())
ax.set_extent([-100, 0, 0, 80], crs=ccrs.PlateCarree())
ax.coastlines()
ax.gridlines(draw_labels=True)
datetime_label = ax.text(-115, 40, start_dt.strftime('%Y-%m-%d %H:%M:%S'), 
    fontsize=10, 
    color="red", 
    transform=ccrs.PlateCarree(), 
    bbox=dict(facecolor="white", alpha=0.5, edgecolor="none")
)
ax.set_title("Hurricane Season 2020")

Were going to iterate over both, the selected storms and the selected drifters. For each of the trajectories, we plot their starting point and store some data to be utilized for generating the animation.

Lets unpack the data variables from a ragged array (1-d array composed of each rows data variable segment) into a list of row data variable segments.

In [ ]:
from clouddrift.ragged import unpack

storm_indices = np.array(range(len(matching_storms.id)))
drifter_indices = np.array(range(len(matching_drifters.id)))

storm_lons = unpack(matching_storms.lon, matching_storms.rowsize, storm_indices)
storm_lats = unpack(matching_storms.lat, matching_storms.rowsize, storm_indices)

drifter_lons = unpack(matching_drifters.lon, matching_drifters["rowsize"], drifter_indices)
drifter_lats = unpack(matching_drifters.lat, matching_drifters["rowsize"], drifter_indices)
drifter_temps = unpack(matching_drifters.temp, matching_drifters["rowsize"], drifter_indices)

Use the unpacked segments and plot the initial starting point of the storm and drifter trajectories (we also do some setup with setting up indexes to make searching later down the line easier).

In [ ]:
storm_lines = list()
drifter_lines = list()

for storm_idx in storm_indices:
    selected_lon, selected_lat = storm_lons[storm_idx], storm_lats[storm_idx]
    selected_lon, selected_lat = selected_lon.set_xindex("time"), selected_lat.set_xindex("time")
    line = ax.plot(selected_lon[0], selected_lat[0],
        linestyle="-", linewidth=3,
        transform=ccrs.PlateCarree(),
    )
    storm_lines.append((selected_lon, selected_lat, line[0]))

for drifter_idx in drifter_indices:
    selected_lon, selected_lat, selected_temp = drifter_lons[drifter_idx], drifter_lats[drifter_idx], drifter_temps[drifter_idx]
    selected_lon, selected_lat, selected_temp = (selected_lon.set_xindex("time"), selected_lat.set_xindex("time"), selected_temp.set_xindex("time"))
    line = ax.plot(selected_lon[0], selected_lat[0],
        linestyle="-", linewidth=1,
        transform=ccrs.PlateCarree(),
    )
    drifter_lines.append((selected_lon, selected_lat, selected_temp, line[0]))

Lets now find the upper and lower bounds for the current temperature values being plotted. We'll use this to create a color map to color the drifter trajectories. 

In [ ]:
import matplotlib.cm as cm
import matplotlib.colors as colors


min_t = np.nanmin([np.nanmin(temp) for (_, _, temp, _) in drifter_lines])
max_t = np.nanmax([np.nanmax(temp) for (_, _, temp, _) in drifter_lines])

cmap = plt.get_cmap("inferno")
norm = colors.Normalize(vmin=min_t, vmax=max_t)

(min_t, max_t, cmap(norm(80)))

Lets add the legend for the color bar.

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size="3%", pad=0.75, axes_class=plt.Axes)
fig.colorbar(cm.ScalarMappable(norm=norm, cmap=cmap), cax=cax, label="temperature (K)")
fig

Lets take the start and end date we've used for the criteria and generate an range of values that each map uniquely to a frame.

In [ ]:
import pandas as pd
frame_count = 500
daterange = pd.date_range(start_dt, end_dt, frame_count)

Lets now generate each frame by selecting the date associated to it which we utilize to update each trajectory with new observations

In [ ]:
storm_frames = list()
drifter_frames = list()
frames = dict()
tail_len = 20

for idx, dt in enumerate(daterange):
    if ((idx+1) % 100 == 0):
        print(f"generating index: {idx}")

    storm_updates = list()
    for s_lon, s_lat, s_line in storm_lines:
        sel_d_lon = s_lon.sel(time=slice(start_dt, dt))
        sel_d_lat = s_lat.sel(time=slice(start_dt, dt))
        storm_updates.append((sel_d_lon, sel_d_lat, s_line))

    drifter_updates = list()
    for d_lon, d_lat, d_temp, d_line, in drifter_lines:
        sel_d_lon = d_lon.sel(time=slice(start_dt, dt))
        sel_d_lat = d_lat.sel(time=slice(start_dt, dt))
        sel_d_temp = d_temp.sel(time=slice(start_dt, dt))
        sel_d_lon = sel_d_lon.tail(obs=tail_len)
        sel_d_lat = sel_d_lat.tail(obs=tail_len)
        sel_d_temp = sel_d_temp.tail(obs=tail_len)
        drifter_updates.append((sel_d_lon, sel_d_lat, sel_d_temp, d_line))

    frames[dt] = dict(drifter_updates=drifter_updates, storm_updates=storm_updates)

We define an update function that, using the frame index, selects the frame and updates each trajectories longitude and latitude (and also temperature for the case of the drifters)

In [ ]:

sorted_dates = sorted(frames.keys())

def update(frame_idx):
    frame_dt = sorted_dates[frame_idx]
    frame = frames[frame_dt]
    drifter_updates = frame["drifter_updates"]
    storm_updates = frame["storm_updates"]

    datetime_label.set_text(frame_dt.strftime('%Y-%m-%d %H:%M:%S'))

    updated_lines = list()
    for x_update, y_update, line in storm_updates:
        line.set_xdata(x_update)
        line.set_ydata(y_update)
        updated_lines.append(line)

    for x_update, y_update, temps, line in drifter_updates:
        line.set_xdata(x_update)
        line.set_ydata(y_update)
        if len(temps) > 0:
            line.set_color(cmap(norm(np.nanmean(temps))))
        updated_lines.append(line)
    return updated_lines

ani = animation.FuncAnimation(fig=fig, func=update, frames=frame_count, interval=50)

Now generate the animation!

In [ ]:
ani.save("storm_drifters.gif", dpi=DPI, progress_callback=lambda i, n: print(f'Saving frame {i}/{n}'))